In [1]:
import ClimaAtmos as CA
import ClimaUtilities
import ClimaUtilities: TimeVaryingInputs
using Dates
import ClimaCoreTempestRemap
import ClimaCore.Fields

In [2]:
cd("../../")

In [3]:
# get simulation object so as to get target_space
config = CA.AtmosConfig("config/model_configs/prognostic_edmfx_gcmdriven_column.yml")
simulation = CA.get_simulation(config)

┌ Info: Loading yaml file config/default_configs/default_config.yml
└ @ ClimaAtmos /Users/julianschmitt/Documents/Research/loss/ClimaAtmos.jl/src/solver/types.jl:702
┌ Info: Loading yaml file config/model_configs/prognostic_edmfx_gcmdriven_column.yml
└ @ ClimaAtmos /Users/julianschmitt/Documents/Research/loss/ClimaAtmos.jl/src/solver/types.jl:702
[ Info: Running on CPUSingleThreaded
[ Info: Setting up single-process ClimaAtmos run
[ Info: Running ClimaCore in unthreaded mode
┌ Info: Making AtmosConfig with config files: 
│    config/default_configs/default_config.yml
└    config/model_configs/prognostic_edmfx_gcmdriven_column.yml
[ Info: Using CAM_SE hyperdiffusion. vorticity_hyperdiffusion_coefficient, scalar_hyperdiffusion_coefficient and divergence_damping_factor in the config will be ignored.
┌ Info: numerics 
│            `energy_upwinding`::`Val{:vanleer_limiter}`
│            `tracer_upwinding`::`Val{:vanleer_limiter}`
│             `edmfx_upwinding`::`Val{:first_order}`
│     `

MethodError: MethodError: no method matching grav(::DateTime)

Closest candidates are:
  grav(!Matched::ClimaAtmos.Parameters.AbstractClimaAtmosParameters)
   @ ClimaAtmos ~/Documents/Research/loss/ClimaAtmos.jl/src/parameters/Parameters.jl:124


In [4]:
Fields.coordinate_field(simulation.integrator.u.c.ρ)

ClimaCore.Geometry.XYZPoint{Float32}-valued Field:
  x: Float32[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
  y: Float32[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
  z: Float32[15.0, 46.4423, 80.9064, 118.68, 160.076, 205.44, 255.145, 309.601, 369.254, 434.591  …  23340.1, 24915.8, 26546.9, 28229.2, 29957.5, 31726.0, 33528.5, 35357.8, 37206.8, 39067.6]

In [5]:
pwd()

"/Users/julianschmitt/Documents/Research/loss/ClimaAtmos.jl"

In [23]:
using NCDatasets
ds = NCDataset("examples/tvforcing/sim_forcing_site23_ng_xyz.nc")

ds["z"][:]
start_time = DateTime("20070701", "yyyymmdd")

time_index = argmin(abs.(ds["time"][:] .- start_time))

(ds["ts"][1, 1, 1, time_index],)[0]

BoundsError: BoundsError: attempt to access Tuple{Float64} at index [0]

In [6]:
start_time = DateTime(2007, 7, 1)
tv = TimeVaryingInputs.TimeVaryingInput(["examples/tvforcing/sim_forcing_site23_ng_xyz.nc"],
                                ["ta"],
                                axes(similar(simulation.integrator.u.c.ρ));
                                reference_date = start_time,
                                regridder_type = :InterpolationsRegridder,
);

In [7]:
import ClimaCore:Fields
F1 = zero(simulation.integrator.u.c.ρ)
next_t = start_time + Dates.Second(100000)
next_t
TimeVaryingInputs.evaluate!(F1, tv, next_t)
F1

Float32-valued Field:
  Float32[297.298, 296.994, 296.662, 296.297, 295.898, 295.461, 294.981, 294.456, 293.896, 293.352  …  214.717, 219.889, 225.916, 227.766, 229.666, 232.685, 237.697, 239.797, 245.012, 251.404]

In [27]:
eltype(Fields.coordinate_field(Fields.level(simulation.integrator.u.c.ρ, 1)))

ClimaCore.Geometry.XYZPoint{Float32}

In [30]:
#length(parent(Fields.coordinate_field(Fields.level(simulation.integrator.u.c.ρ, 1)).z)[:]) ==1
axes(similar(Fields.level(simulation.integrator.u.c.ρ, 1), FT))

SpectralElementSpace2D:
  quadrature: 1-point Gauss-Legendre quadrature

In [14]:
import ClimaCore: Fields
FT = Float32
tv = TimeVaryingInputs.TimeVaryingInput(["examples/tvforcing/sim_forcing_site23_ng_xyz.nc"],
                                ["coszen"],
                                axes(similar(Fields.level(simulation.integrator.u.c.ρ, 1), FT));
                                reference_date = start_time,
                                regridder_type = :InterpolationsRegridder,
);
F1 = zero(Fields.level(simulation.integrator.u.c.ρ, 1))
next_t = start_time + Dates.Second(3600 * 1)
next_t
TimeVaryingInputs.evaluate!(F1, tv, next_t)
F1

Float32-valued Field:
  Float32[0.735423]